In [42]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [43]:
df = pd.read_csv('./movies_metadata.csv')
# [19730, 29503, 35587]
df = df.drop([19730, 29503, 35587])

/tmp/ipykernel_71894/2299916687.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./movies_metadata.csv')


Узнаем, какие данные есть в датасете

In [44]:
df = df.convert_dtypes()
df.dtypes

adult                    string[python]
belongs_to_collection    string[python]
budget                   string[python]
genres                   string[python]
homepage                 string[python]
id                       string[python]
imdb_id                  string[python]
original_language        string[python]
original_title           string[python]
overview                 string[python]
popularity                       object
poster_path              string[python]
production_companies     string[python]
production_countries     string[python]
release_date             string[python]
revenue                           Int64
runtime                           Int64
spoken_languages         string[python]
status                   string[python]
tagline                  string[python]
title                    string[python]
video                           boolean
vote_average                    Float64
vote_count                        Int64
dtype: object

In [45]:
df['id'] = df['id'].astype(int)

In [46]:
df.isnull().any()

adult                    False
belongs_to_collection     True
budget                   False
genres                   False
homepage                  True
id                       False
imdb_id                   True
original_language         True
original_title           False
overview                  True
popularity                True
poster_path               True
production_companies      True
production_countries      True
release_date              True
revenue                   True
runtime                   True
spoken_languages          True
status                    True
tagline                   True
title                     True
video                     True
vote_average              True
vote_count                True
dtype: bool

Удаляем следующие столбцы:
* `adults` - поскольку в дальнейшем будет добавлен столбец в возвратным рейтингом фильма MPAA
* `belongs_to_collection`
* `homepage`
* `popularity` - эта метрика [высчитывается](https://developer.themoviedb.org/docs/popularity-and-trending) для фильма каждый день, поэтому она актуальна в тот момент, когда были собраны данные в датасете
* `poster_path`
* `video`

In [47]:
df = df.drop(columns=['adult', 'belongs_to_collection', 'homepage', 'popularity', 'poster_path', 'video'], axis=1)

Посмотрим, у скольки записей есть целевая переменная: сборы (`revenue`)

In [48]:
df[(df['revenue'].notna()) & (df['revenue'] != 0)].shape

(7408, 18)

Добавляем в исходный датасет данные, полученные с сайта Box Office Mojo.

In [49]:
df_mojo = pd.read_csv('./scrapper/result.csv')
df_mojo.head()

,imdb_id,domestic_distributor,domestic_opening,budget,mpaa,running_time,genres,domestic_revenue,international_revenue,worldwide_revenue
0,tt0113041,Walt Disney Studios Motion Pictures,11134978,0,PG,1 hr 46 min,Comedy Family Romance,76594107,0,76594107
1,tt0113228,Warner Bros.,7797185,0,PG-13,1 hr 41 min,Comedy Romance,71518503,0,71518503
2,tt0112760,Metro-Goldwyn-Mayer (MGM),2371415,98000000,PG-13,2 hr 4 min,Action Adventure Comedy,10017322,0,10017322
3,tt0114576,Universal Pictures,4782445,0,R,1 hr 51 min,Action Crime Thriller,20350171,44000000,64350171
4,tt0112453,Universal Pictures,1519755,0,NaN,1 hr 18 min,Adventure Animation Drama Family History,11348324,766,11349090


Удаляем поля, которые уже есть в исходном датасете.
`running_time` можно удалять, т.к. в исходных данных он проставлен практическиу всех фильмов.

In [50]:
df_mojo = df_mojo.drop(['domestic_revenue', 'international_revenue', 'running_time', 'genres'], axis=1)
df_mojo.rename(columns={'budget': 'scrapped_budget'}, inplace=True)

In [51]:
df = df.merge(df_mojo, on=['imdb_id'], how='inner')

Дополняем инфоормацию о бюджете и сборах исходного датасета с помощью данных с Box Office Mojo

In [52]:
def fill_zero_fileds(field_1, field_2):
    def f(row):
        if row[field_1] != 0:
            return row[field_1]
        return row[field_2]
    
    return f

df['new_budget'] = df.apply(fill_zero_fileds('budget', 'scrapped_budget'), axis=1)
df['new_revenue'] = df.apply(fill_zero_fileds('revenue', 'worldwide_revenue'), axis=1)
df = df.drop(columns=['budget', 'scrapped_budget', 'revenue', 'worldwide_revenue'], axis=1)
df = df.rename(columns={'new_budget': 'budget', 'new_revenue': 'revenue'})

In [53]:
df[df['revenue'] != 0].shape

(20260, 21)

Как видно, количество "валидных" записей увеличилось с 7 тыс. до 20 тыс.

Удалим строки, в которых неизвестна целевая переменная: сборы (`revenue`)

In [54]:
df = df[(df['revenue'].notna()) & (df['revenue'] != 0)]

Поскольку столцбы `genres`, `production_companies`, `production_countries` и `spoken_languages` представляют собой список словарей, нужно их обработать отдельно.

In [55]:
def get_value_from_dict(column, key):
    def f(row):
        dicts = literal_eval(row[column])
        res = []
        for c in dicts:
            res.append(c[key])
        if len(res) == 0:
            return np.NAN
        return res

    return f

df['genres'] = df.apply(get_value_from_dict('genres', 'name'), axis=1)
df['production_companies'] = df.apply(get_value_from_dict('production_companies', 'name'), axis=1)
df['production_countries'] = df.apply(get_value_from_dict('production_countries', 'iso_3166_1'), axis=1)
df['spoken_languages'] = df.apply(get_value_from_dict('spoken_languages', 'iso_639_1'), axis=1)

Теперь нужно добавить информацию об актерах и режиссере

In [56]:
credit_df = pd.read_csv('./credits.csv')

def get_3_actors(row):
    cast = literal_eval(row['cast'])
    cast = sorted(cast, key=lambda x: x['order'])
    cast = list(map(lambda x: x['name'], cast))
    return cast[:3]

def get_director(row):
    crew = literal_eval(row['crew'])
    for c in crew:
        if c['job'] == 'Director':
            return c['name']
    return np.NAN

credit_df['actors'] = credit_df.apply(get_3_actors, axis=1)

credit_df['director'] = credit_df.apply(get_director, axis=1)

credit_df = credit_df.drop(columns=['cast', 'crew'], axis=1)

df = df.merge(credit_df, on=['id'], how='inner')

In [59]:
df.to_csv('./movie_dataset.csv')